# 🚘 Analyzing SF traffic stops with `Python`: Part 1

<img src="img/sf-traffic.jpg" alt="traffic" width="600" align="left"/>

This is Part 1. Other parts can be found [here](https://www.jdgrossman.com).

## Introduction

In this series of tutorials, we'll use `Python` to explore traffic stops in San Francisco (SF). In particular, we'll investigate whether there is evidence of racial discrimination in SF's policing practices. 

> **Important note**: Policing can be a sensitive subject. It's important to remember that each row in our data represents a real interaction between a police officer and driver. Please keep this in mind as you work through the tutorial, and be sure to engage with the material to the extent you're comfortable. 

By the end of the tutorials, you'll have foundational understanding of the following:
1. 📊 How to use `Python` to explore tabular data and calculate descriptive statistics. 
2. 📈 How to make an informative plot with `Python`
2. ⚖️ How to approach questions about social policy with data. 

Let's get started!

## ✅ Set up

While the core `Python` language contains many useful functions (e.g., `sum` and `len`), there is vast functionality built on top of `Python` by community members.

Make sure to run the cell below. It imports additional useful functions and loads in data. 

In [1]:
# Load in additional functions for manipulating data from the Pandas package
import pandas as pd
from numpy import floor

# This is where the data is stored.
STOPS_PATH = "https://github.com/joshuagrossman/dsb-win-2023/raw/main/opp-munging-plotting/data/sf_stop_data.feather"

# Read in the data
stops = pd.read_feather(STOPS_PATH)
stops['date'] = pd.to_datetime(stops.date)

### 🖼️ The data frame

Data frames are like spreadsheets in Microsft Excel or Google Sheets: they have rows and columns, and each cell in the spreadsheet contains data.

Run the cell below to preview the `stops` data. What do you notice?

> 🔎 The `head()` method allows us to see the first couple rows of a dataframe.

In [2]:
stops.head()

,date,time,location,lat,lng,district,age,race,gender,arrested,contraband_found,searched,reason_for_stop
0,2009-01-01,10:10:00,1736 PALOU,37.735669,-122.393353,C,22.0,black,female,False,False,True,Equipment violation
1,2009-01-01,10:15:00,THRIFT/PLYMTH,37.717494,-122.456140,I,44.0,black,male,False,None,False,Moving violation
2,2009-01-01,10:20:00,FLORIDA/19TH,37.760440,-122.410986,D,45.0,white,female,False,None,False,Equipment violation
3,2009-01-01,10:20:00,19TH AVE/MORAGAE,37.756110,-122.476643,I,27.0,white,male,False,None,False,Equipment violation
4,2009-01-01,10:36:00,19TH/LINCOLN,37.765483,-122.477397,I,29.0,white,male,False,None,False,Equipment violation


⬆️ From the preview above, we might guess that each row in the `stops` dataframe represents a stop, and each column contains information about each stop.

> This guess is correct!

### 💭 Asking questions about the data

As an analyst, you might start with some basic questions:

1. How many stops (i.e., rows) are in the `stops` data?
2. What do we know about each stop?
3. When was the earliest stop?
4. What were the most commons reasons for stops?
5. Who is most likely to get stopped?

Let's start with the first question: how many rows are in the `stops` data?

In [3]:
len(stops)

636161

Looks like we have information on approximately 640,000 stops.

The `shape` attribute provides the number of rows and columns:

In [4]:
stops.shape

(636161, 13)

What do we know about each stop?

In [5]:
stops.columns

Index(['date', 'time', 'location', 'lat', 'lng', 'district', 'age', 'race',
       'gender', 'arrested', 'contraband_found', 'searched',
       'reason_for_stop'],
      dtype='object')

It looks like we have the basics of each stop: time, location, demographics, and outcomes.

## 🚀 Exercise: Stop dates

When did the traffic stops in the `stops` data occur? 

Use the `date` column in the `stops` data to get a sense of when stops typically occur. Write a comment explaining your results. 

A few pointers:

- 💵 To extract a column from a data frame, use `[]` or `.col_name`. 

> To retrieve column `age` from data frame `df`, we write `df['age']` or `df.age`.

> Columns of `Pandas` data frames are stores as `Series` objects. For now, you can think of them like a list.

- You may find the following `Series` methods helpful: `min()`, `max()`, and `sample()`. 

> You can learn more about a method `f` of object `x` by running `help(x.f)`.

In [6]:
## Your code here!

# START

date_col = stops['date']

# alternatively
data_col = stops.date

# The earliest stop took place on Jan 1, 2009.
print(date_col.min())

# The last stop took place on Jun 30, 2016.
print(date_col.max())

# We observe stops between 2009 and 2016. You technically
# can't confirm this using just min and max!
print(date_col.sample(10))

# END

2009-01-01 00:00:00
2016-06-30 00:00:00
152821   2010-05-28
356828   2012-06-17
161307   2010-07-01
268260   2011-07-05
545138   2015-05-16
355195   2012-06-09
604485   2016-02-10
359743   2012-07-02
372526   2012-08-28
550836   2015-06-10
Name: date, dtype: datetime64[ns]


## 📝 Adding new columns with `[]` and `assign()`

Our data extends from 2009 to the first half of 2016. Suppose want to examine the most recent full year of data: 2015.

Problem: We don't have a `year` column. We can create a new column with `[]` or the `assign()` method.

🖥️ Usage: `df['new_col_name'] = f( df['existing_col_name'] )`
* `df`: the data frame
* `new_col_name`: name of the new column to add
* `f`: function to apply to existing column(s) to generate the new column
* `existing_col_name`: name of existing column

For example, here's how we could add a column to `stops` containing the first digit of the driver's age.

In [7]:
stops['age_first_digit'] = floor(stops['age']/10)

stops['age_first_digit'].head()

0    2.0
1    4.0
2    4.0
3    2.0
4    2.0
Name: age_first_digit, dtype: float64

Alternatively, we can use the `assign()` method:

In [8]:
# assign() applies the anonymous function directly to the data.
# We can chain Pandas methods to make our code more readable
# The outer parentheses allow us to avoid using a continuation character `\`
# at the end of each line.
(
    stops
    .assign(age_first_digit = lambda df: floor(df['age']/10))
    .head()
)

,date,time,location,lat,lng,district,age,race,gender,arrested,contraband_found,searched,reason_for_stop,age_first_digit
0,2009-01-01,10:10:00,1736 PALOU,37.735669,-122.393353,C,22.0,black,female,False,False,True,Equipment violation,2.0
1,2009-01-01,10:15:00,THRIFT/PLYMTH,37.717494,-122.456140,I,44.0,black,male,False,None,False,Moving violation,4.0
2,2009-01-01,10:20:00,FLORIDA/19TH,37.760440,-122.410986,D,45.0,white,female,False,None,False,Equipment violation,4.0
3,2009-01-01,10:20:00,19TH AVE/MORAGAE,37.756110,-122.476643,I,27.0,white,male,False,None,False,Equipment violation,2.0
4,2009-01-01,10:36:00,19TH/LINCOLN,37.765483,-122.477397,I,29.0,white,male,False,None,False,Equipment violation,2.0


❗❗❗Important note❗❗❗: Unless otherwise specific, `Pandas` functions and methods are "copy on modify". In other words, when we apply a function or method to data, `Pandas` creates a copy of the data and then modifies the copy. The original data is unchanged.

So, `assign()` alone will not change the original data. 

### 🚀 Exercise

1. Use add a new column called `yr` to our `stops` data. The `Series` attribute `dt.year` may come in handy.

> You can read about the `dt.year` attribute by running `help(Series.dt.year)`.

2. Assign the resulting data frame to a new variable called `stops_w_yr`. 
 
3. Finally, run `stops_w_yr['yr'].value_counts()`. 

> What do you think `value_counts()` does? Do you notice any patterns?

In [9]:
# Your code here!

# START

stops_w_yr = (
    stops
    .assign(yr = lambda df: df['date'].dt.year)
)

# Count the number of stops in each year.
# There are fewer stops in later years.
# There are a lot fewer stops in 2014 and 2016.
# We saw early that we only get the first half of 2016.
# But what's going on with 2014?
stops_w_yr['yr'].value_counts()

# END

2009    110269
2010    104254
2011     99476
2015     85689
2012     82362
2013     74144
2016     40215
2014     39752
Name: yr, dtype: int64

## 📝 Selecting rows with `[]` and `query()`

Now that we have a `yr` column, we want to limit our data to just the stops in 2015.

Problem: We have data from 2009 to 2016. To limit to specific rows, we can use `[]` or the `query()` method.

🖥️ Usage: `data[condition]`
* `data`: the data frame
* `condition`: an array-like object the same length as `date` where TRUE indicates the rows in `data` to keep.

For example, here's how we could limit `stops` to drivers under 30 years old:

In [10]:
stops[stops['age'] < 30].head()

,date,time,location,lat,lng,district,age,race,gender,arrested,contraband_found,searched,reason_for_stop,age_first_digit
0,2009-01-01,10:10:00,1736 PALOU,37.735669,-122.393353,C,22.0,black,female,False,False,True,Equipment violation,2.0
3,2009-01-01,10:20:00,19TH AVE/MORAGAE,37.756110,-122.476643,I,27.0,white,male,False,None,False,Equipment violation,2.0
4,2009-01-01,10:36:00,19TH/LINCOLN,37.765483,-122.477397,I,29.0,white,male,False,None,False,Equipment violation,2.0
7,2009-01-01,10:44:00,19TH/SANTIAGO,37.744918,-122.475846,I,29.0,white,female,False,None,False,Equipment violation,2.0
11,2009-01-01,10:55:00,LA SALLE @ NEWCOMB,37.733112,-122.385115,C,26.0,black,male,False,False,True,Moving violation,2.0


We can alternatively use the `query()` method:

In [11]:
stops.query('age < 30').head()

,date,time,location,lat,lng,district,age,race,gender,arrested,contraband_found,searched,reason_for_stop,age_first_digit
0,2009-01-01,10:10:00,1736 PALOU,37.735669,-122.393353,C,22.0,black,female,False,False,True,Equipment violation,2.0
3,2009-01-01,10:20:00,19TH AVE/MORAGAE,37.756110,-122.476643,I,27.0,white,male,False,None,False,Equipment violation,2.0
4,2009-01-01,10:36:00,19TH/LINCOLN,37.765483,-122.477397,I,29.0,white,male,False,None,False,Equipment violation,2.0
7,2009-01-01,10:44:00,19TH/SANTIAGO,37.744918,-122.475846,I,29.0,white,female,False,None,False,Equipment violation,2.0
11,2009-01-01,10:55:00,LA SALLE @ NEWCOMB,37.733112,-122.385115,C,26.0,black,male,False,False,True,Moving violation,2.0


### 🚀 Exercise

1. Use `[]` or `query()` to filter the `stops` data to just 2015. Assign the result to a variable called `stops_2015`.

2. In the previous exercise, we saw that there were a lot fewer stops in 2014 than expected. Figure out why.

3. For practice, filter to stops occurring in 2013 or 2014 among female drivers less than 30 years old or more than 60 years old. 

In [12]:
# Your code here!

# START

# 1.
stops_2015 = stops_w_yr[stops_w_yr.yr==2015]

# 1., alternative
stops_2015 = stops_w_yr.query('yr==2015')

# 2.
stops_2014 = stops_w_yr.query('yr==2014')

# 2014 only covers stops before May 31
print(stops_2014.date.min())
print(stops_2014.date.max())

# 3. 
(
    stops_w_yr
    .query("""
        (yr == 2013 | yr == 2014) &\
        (gender == 'female') &\
        (age < 30 | age > 60)   
    """)
    .head()
)
# END

2014-01-01 00:00:00
2014-05-30 00:00:00


,date,time,location,lat,lng,district,age,race,gender,arrested,contraband_found,searched,reason_for_stop,age_first_digit,yr
396367,2013-01-01,01:02:00,17 ST / SAN BRUNO AV,37.764619,-122.405486,C,19.0,hispanic,female,False,None,False,Equipment violation,1.0,2013
396369,2013-01-01,10:26:00,3RD ST & BANCROFT,37.726216,-122.393834,C,22.0,black,female,False,None,False,Equipment violation,2.0,2013
396370,2013-01-01,10:35:00,6TH ST & HOWARD,37.779741,-122.407175,B,28.0,black,female,False,None,False,Equipment violation,2.0,2013
396375,2013-01-01,11:45:00,SUTTER & STOCKTON,37.789432,-122.406993,A,22.0,white,female,False,None,False,Equipment violation,2.0,2013
396381,2013-01-01,12:45:00,SOUTH VANNESS & MARKET,37.775122,-122.419335,B,24.0,hispanic,female,False,None,False,Equipment violation,2.0,2013


## 📝 Aggregating data with `agg()`

What was the average, median, maximum, and minimum age of drivers in 2015?

Problem: We want to aggregate the values in a column. To do this, we use `agg()`.

In [13]:
# Old method.
print(stops_2015.age.mean())
print(stops_2015.age.median())
print(stops_2015.age.max())
print(stops_2015.age.min())

# New method!
(
    stops_2015
    .agg({
        'age' : ['mean', 'median', 'max', 'min']
    })
)

38.81238545496994
36.0
99.0
10.0


,age
mean,38.812385
median,36.000000
max,99.000000
min,10.000000


Neat! But, it's not groundbreaking. `agg()` really ☀️ shines ☀️ when used with `groupby()`.

## 📝 Getting powerful with `groupby()` and `agg()`

Here's where things get really interesting. The techniques in this section account for a **huge** chunk of most data science workflows. 

Suppose I'm interested in the average age of drivers in each district.

> `set(l)` returns the set of unique values in a list `v`

In [14]:
set(stops_2015.district)

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'}

You already have the tools to find the average age of drivers by district! 

Looks a little scary though...

In [15]:
print(stops_2015.query('district=="A"').age.mean())
print(stops_2015.query('district=="B"').age.mean())
print(stops_2015.query('district=="C"').age.mean())
print(stops_2015.query('district=="D"').age.mean())
print(stops_2015.query('district=="E"').age.mean())
print(stops_2015.query('district=="F"').age.mean())
print(stops_2015.query('district=="G"').age.mean())
print(stops_2015.query('district=="H"').age.mean())
print(stops_2015.query('district=="I"').age.mean())
print(stops_2015.query('district=="J"').age.mean())

39.258339510748705
38.141873278236915
36.60643968236702
37.18025114155251
38.753412969283275
39.789667896678964
40.831689868522815
38.23528735632184
39.80102989558003
38.828421052631576


# 😓

We now know the average age in each district, but there are some issues:
- We had to write a lot of repeated code.
- What if there were 100 districts? Or 1,000,000 districts?
- The results aren't labeled. We'd have to write even more code to label the output.

Here's another way to answer the question, but with less code:

In [16]:
(
    stops_2015
    .groupby('district')
    .agg({'age' : 'mean'})
)

,age
district,
A,39.258340
B,38.141873
C,36.606440
D,37.180251
E,38.753413
F,39.789668
G,40.831690
H,38.235287
I,39.801030


# 😮

The next section will explain the magic of grouping.

### 📝 The mechanics of `groupby()`

It's **very** common to calculate an aggregate statistic (e.g., `sum` or `mean`) for different groups (e.g., district or class year).

The *split-apply-combine* paradigm handles these situations:
- **Split** the data by group into mini-datasets
- **Apply** a function to each mini-dataset
- **Combine** the mini-datasets back together

🖼️ A visual:

<img src="img/split-apply-combine.drawio.png" alt="splitapplycombine" width="600" align="left"/>

#### 📝 Splitting with `groupby`

`groupby` handles the *splitting* step.

Problem: The data isn't grouped. To split the data, we use `groupby`.

🖥️ Usage: `data.groupby('column_name')`
* `data`: the data frame
* `column_name`: the name of the column to group by.

Let's try grouping the `stops` data by district.

In [17]:
stops_2015_grouped = stops_2015.groupby('district')

stops_2015_grouped

Something seems to have happened in the background?

#### 📝 Applying and combining with `agg()`

`agg()` *applies* an aggregating function to each mini-dataset created by `groupby`. It then *combines* the mini-datasets.

We've already seen `agg()` applied to the ungrouped data'.

> You can think of ungrouped data as grouped data with just one group for the entire dataset

In [18]:
stops_2015.agg({'age' : 'mean'})

age    38.812385
dtype: float64

Let's try `agg()` with the data grouped by district.

In [19]:
stops_2015_grouped.agg({'age' : 'mean'})

,age
district,
A,39.258340
B,38.141873
C,36.606440
D,37.180251
E,38.753413
F,39.789668
G,40.831690
H,38.235287
I,39.801030


If we want to give the aggregated columna new name, we can use a tuple: `(col_to_aggregate, aggregating_function)`

In [20]:
stops_2015_grouped.agg(mean_age = ('age', 'mean'))

,mean_age
district,
A,39.258340
B,38.141873
C,36.606440
D,37.180251
E,38.753413
F,39.789668
G,40.831690
H,38.235287
I,39.801030


### 🚀 Exercise

1. Use `groupby()` and `agg()` to calculate, by district, (1) the number of stops, (2) the proportion of stops that resulted in a search, and (3) the proportion of **searches** (not stops) that resulted in contraband found. What can you conclude from the results?

> You might find the `count` aggregating function useful.

2. Redo part 1, but group by race instead of district. What do you conclude from the result?

3. Redo part 1, but group by district **and** race. What is your interpretation of the results?

> You can pass a list of columns to `groupby()` to group by multiple columns.

In [21]:
# Your code here!

# START

# 1.
# Search and contraband rates differ by district.
# Districts with higher search rates tend to have lower contraband rates,
# and vice-versa. 
print(
    stops_2015
    .groupby('district')
    .agg(
        n_stops = ('district', 'count'),
        
        n_searches = ('searched', 'sum'),
        
        n_contraband = ('contraband_found', 'sum'),
    )
    .assign(
        search_rate = lambda df: df.n_searches/df.n_stops,
        contraband_rate = lambda df: df.n_contraband/df.n_searches
    )
)

# 2.
# Search rates for Black and Hispanic drivers are higher than drivers of 
# other race/ethnicity groups. However, the contraband rates for Black and Hispanic
# drivers are lower than other race/ethnicities. Perhaps the search threshold
# for Black and Hispanic drivers is lower, so the Black and Hispanic drivers
# who are stopped are less "risky"?
print(
    stops_2015
    .groupby('race')
    .agg(
        n_stops = ('district', 'count'),
        
        n_searches = ('searched', 'sum'),
        
        n_contraband = ('contraband_found', 'sum'),
    )
    .assign(
        search_rate = lambda df: df.n_searches/df.n_stops,
        contraband_rate = lambda df: df.n_contraband/df.n_searches
    )
)

# # 3.
# # The output is quite long, but it looks like the gaps in search and contraband
# # rates for Black and Hispanic drivers persist across districts, though
# # the gaps differ in size across districts.
# # A plot might help us digest these results more easily.
print(
    stops_2015
    .groupby(['district', 'race'])
    .agg(
        n_stops = ('district', 'count'),
        
        n_searches = ('searched', 'sum'),
        
        n_contraband = ('contraband_found', 'sum'),
    )
    .assign(
        search_rate = lambda df: df.n_searches/df.n_stops,
        contraband_rate = lambda df: df.n_contraband/df.n_searches
    )
)

# END

          n_stops  n_searches n_contraband  search_rate contraband_rate
district                                                               
A            8098         185           53     0.022845        0.286486
B            7988         343           77     0.042939         0.22449
C            9199        1163          140     0.126427        0.120378
D            8760         584          110     0.066667        0.188356
E            5862         393           57     0.067042        0.145038
F            7049         102           45     0.014470        0.441176
G           10345         119           35     0.011503        0.294118
H            8702         569           75     0.065387         0.13181
I           13983         214           61     0.015304        0.285047
J            5703         515          115     0.090303        0.223301
                        n_stops  n_searches n_contraband  search_rate  \
race                                                           

## Concluding remarks

The method used in the final exercise is called an **outcome test**. Someone actually won a Nobel Prize for this kind of work! 

Here's what we'll do in Part 2:
- Use 📊plots📈 to reduce the cognitive burden of reading long tables.
- Learn how to combine data from multiple sources
- Dig deeper into our results. Can we say anything about racial/ethnic discrimination based on our results? What additional tests can we conduct? How can we clearly present our findings?
